<a href="https://colab.research.google.com/github/PhilipMathieu/unet-orthoimagery/blob/main/colab_augment_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb
!rm -r sample_data
!git clone https://github.com/PhilipMathieu/unet-orthoimagery.git ./repo/

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 13.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.8/198.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.6 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=90959e37b6cb3f9a7d5b4937c65c1e595f6e6780322480a04465cc1d95ed5107
  Stored in directory: /root/.cache/pip/wheels/b7/0a/67/ada2a22079218c75a88361c0782855cc72aebc4d18d0289d05
Successfully built pathtools
Cloning into './repo'...
remote: Enumerating objects: 109103, done.
remote: Counting objects: 100% (4278/4278), done.
remote: Compressing objects: 100% (2586/2586), done.
remote: Total 109103 (delta 1712), reused 4255 (delta 1691), pack-reused 1048

In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
import sys
sys.path.append('./repo/src/')

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from dataAugmentation import main

/content/./repo/src/dataAugmentation.py:26: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
main("repo/data/Image_Chips_128_overlap_balanced_dem/", "AD/")

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [ ]:
import logging
import torch
from train import train_model
from unet.unet_model import UNet

In [ ]:
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
logging.info(f'Using device: {device}')
print(device)

cuda


In [ ]:
model = UNet(n_channels=4, n_classes=1, bilinear=False)
model = model.to(memory_format=torch.channels_last)

In [ ]:
logging.info(f'Network:\n'
                 f'\t{model.n_channels} input channels\n'
                 f'\t{model.n_classes} output channels (classes)\n'
                 f'\t{"Bilinear" if model.bilinear else "transposed conv"} upscaling')


model.to(device=device);

In [ ]:
data_dir = "AD/"

In [ ]:
def train():
    train_model(
        model=model,
        epochs=20,
        device=device,
        data_dir=data_dir,
        learning_rate=1e-2,
        batch_size=128,
        img_scale=1,
        debug=False
    )

try:
    train()
except torch.cuda.OutOfMemoryError: # Giving me syntax error saying '"OutOfMemoryError" is not a valid exception class.'
    logging.error('Detected OutOfMemoryError! '
                    'Enabling checkpointing to reduce memory usage, but this slows down training. '
                    'Consider enabling AMP (--amp) for fast and memory efficient training')
    torch.cuda.empty_cache()
    model.use_checkpointing()
    train()

  0%|          | 0/2560 [00:00<?, ?it/s]

Epoch 1/20:   0%|          | 0/1708 [00:00<?, ?img/s]

Validation round:   0%|          | 0/6 [00:00<?, ?batch/s]

Validation round:   0%|          | 0/6 [00:00<?, ?batch/s]

Validation round:   0%|          | 0/6 [00:00<?, ?batch/s]

...

Validation round:   0%|          | 0/6 [00:00<?, ?batch/s]

Validation round:   0%|          | 0/6 [00:00<?, ?batch/s]

Validation round:   0%|          | 0/6 [00:00<?, ?batch/s]

Validation round:   0%|          | 0/6 [00:00<?, ?batch/s]

Validation round:   0%|          | 0/6 [00:00<?, ?batch/s]

In [ ]:
wandb.finish()

BCE Loss,█▇▄▆▅▆▄▂▃▆▄▅▁▂▂▃▃▂▂▅▁▄▂▃▃▃▁▅▁▂▅▂▂▆▂▂▂▃▄▃
Dice Loss,█▇▄▄▃▄▄▂▂▃▃▃▂▁▃▂▃▁▂▄▁▂▁▂▃▄▁▃▁▂▄▂▂▃▂▁▂▃▃▂
epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇████
learning rate,███▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train loss,█▇▄▅▄▅▄▂▃▅▃▄▁▂▂▂▃▁▂▄▁▃▂▃▃▃▁▄▁▂▄▂▂▅▂▁▂▃▄▃
validation Dice,▂▂▁▃▅▅▆▇███▇████████████████████████████
BCE Loss,0.36951
Dice Loss,0.22161
epoch,20
learning rate,0.0
